In [1]:
# import os
# from neo4j import GraphDatabase as graphdb
# from trengx import graph

import os
from dotenv import load_dotenv
from neo4j import GraphDatabase as graphdb
import sys
sys.path.append('../trengx')
import graph

In [2]:
# .env secrets
uri = os.getenv('NEO4J_URI') # Get the value of the uri variable
user = os.getenv('NEO4J_USER') # Get the value of the user variable
password= os.getenv('NEO4J_PASSWORD')  # Get the value of the pw variable

# Create a Graph class instance
g = graph.Graph(graphdb, uri, user, password)


In [44]:
g.delete_all()

# y_est[i] = w_est[i]*y[i-1]
w_est_i = g.add_node ('num', 'w_est_i', {'value': 1})
w_est_i_id = w_est_i['node_id']
y_im1 = g.add_node ('num', 'y_im1 ', {'value': 1})
y_im1_id = y_im1['node_id']
mul = g.add_node ('op', '*', {'reverse':False})
mul_id = mul['node_id']
print (mul_id)
g.add_edge ('num2op', w_est_i_id, mul_id, {'trigger':False})
g.add_edge ('num2op', y_im1_id, mul_id, {'trigger':True})
y_est_i = g.add_node ('num', 'y_est_i', {'value': 1})
y_est_i_id = y_est_i['node_id']
print (y_est_i_id)
g.add_edge ('op2num', mul_id, y_est_i_id, {'trigger':False})

# e[i] = y[i] - y_est[i]
y_i = g.add_node ('num', 'y_i', {'value': 1.0})
y_i_id = y_i['node_id']
sub = g.add_node ('op', '-', {'reverse':True})
sub_id = sub['node_id']
g.add_edge ('num2op', y_i_id, sub_id, {'trigger':False})
g.add_edge ('num2op', y_est_i_id, sub_id, {'trigger':True})
e_i = g.add_node ('num', 'e_i', {'value': 2})
e_i_id = e_i['node_id']
g.add_edge ('op2num', sub_id, e_i_id)

'''result = g.set_node_value(y_im1_id, 8)
print (result)
result = g.get_node_prop(e_i_id, 'value')
print (result)
g.close()
'''

4
5


"result = g.set_node_value(y_im1_id, 8)\nprint (result)\nresult = g.get_node_prop(e_i_id, 'value')\nprint (result)\ng.close()\n"

In [92]:
query = """
MATCH (in1) WHERE id(in1) = $node_id
CALL apoc.path.expand(in1, "num2op|op2num>", "num|op", 0, -1) 
YIELD path
WITH nodes(path) AS node_sets
ORDER BY size(node_sets) DESC
LIMIT 1 
WITH node_sets as nodes
UNWIND range(0, size(nodes)-2, 2) AS i
WITH nodes[i] AS in1, nodes[i+1] AS op, nodes[i+2] AS out
return in1, op, out
"""


params = {"node_id": y_im1_id}  
result = g.run_query(query, params)
result

[{'in1': {'name': 'y_im1 ', 'value': 1},
  'op': {'name': '*', 'reverse': False},
  'out': {'name': 'y_est_i', 'value': 1}},
 {'in1': {'name': 'y_est_i', 'value': 1},
  'op': {'name': '-', 'reverse': True},
  'out': {'name': 'y_i', 'value': 1.0}}]

In [89]:
query = """
MATCH (in1) WHERE id(in1) = $node_id
CALL apoc.path.expand(in1, "num2op|op2num>", "num|op", 0, -1) 
YIELD path
WITH nodes(path) AS node_sets
ORDER BY size(node_sets) DESC
LIMIT 1 
WITH node_sets as nodes
UNWIND range(0, size(nodes)-2, 2) AS i
WITH nodes[i] AS in1, nodes[i+1] AS op, nodes[i+2] AS out
return in1, op, out
"""

In [ ]:
CALL apoc.path.expand(in1, "num2op|op2num>", "num|op", 0, -1) 
YIELD path
WITH nodes(path) AS nodes
ORDER BY size(nodes) DESC
LIMIT 1
UNWIND range(0, size(nodes)-2, 2) AS i
WITH nodes[i] AS in1, nodes[i+1] AS op, nodes[i+2] AS in2, nodes[i+3] AS out
WHERE (in1)-[:num2op]->(op)-[:op2num]->(out)
CALL apoc.do.case(
  [
    op.name = 'add', 'SET out.value = in1.value + in2.value',
    op.name = 'subtract' AND op.reverse IS NULL, 'SET out.value = in1.value - in2.value',
    op.name = 'subtract' AND op.reverse = true, 'SET out.value = in2.value - in1.value',
    op.name = 'multiply', 'SET out.value = in1.value * in2.value',
    op.name = 'divide' AND op.reverse IS NULL, 'SET out.value = in1.value / in2.value',
    op.name = 'divide' AND op.reverse = true, 'SET out.value = in2.value / in1.value'
  ],
  'RETURN out.value'
) YIELD value
WITH out, value.value AS newVal
SET out.value = newVal
RETURN out
